In [1]:
!pip install PyMuPDF
!pip install camelot-py[cv] tabula-py
!apt install ghostscript python3-tk
import camelot
import fitz
import PIL
from pandas.core.arrays.sparse import array
import pandas as pd
import numpy as np
import itertools
import re
import json

     |████████████████████████████████| 8.8 MB 4.5 MB/s 
     |████████████████████████████████| 40 kB 25 kB/s 
     |████████████████████████████████| 12.0 MB 8.0 MB/s 
     |████████████████████████████████| 5.6 MB 38.2 MB/s 
     |████████████████████████████████| 80 kB 7.3 MB/s 
     |████████████████████████████████| 11.7 MB 27.5 MB/s 
     |████████████████████████████████| 3.6 MB 28.2 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-tk is already the newest version (3.6.9-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono gsfonts libcupsfilters1 libcupsimage2
  libgs9 libgs9-common libijs-0.35 libjbig2dec0 poppler-data
Suggested packages:
  fonts-noto ghostscript-x poppler-utils fonts-japanese-mincho
  | fonts-ipafont-mincho fonts-japan

In [2]:
tables = camelot.read_pdf("VB_T1.pdf", flavor='stream', table_areas=['50,500,620,400'])
camelot.plot(tables[0], kind='contour').show()
tables[0].df

0           1
0                      Het door u te betalen bedrag:  € 9.951,76
1  O\nop het rekeningnummer: BE42  679 2 00 00 0 054            
2  O\nmet de gestructureerde mededeling: +++200/4...            
3                       O\nten laatste op 13.01.2021

In [4]:
#Deze code kan aanslagbiljet-documenten lezen
import re
table = tables[0].df
aanslagbiljet_info = {}
table = pd.DataFrame(pd.Series(table.values.tolist()).str.join(""))
#helper methods
pay_receive_by_index = [row.split("€")[0].strip() for row in table.loc[:,0] if "€" in row][0]
amount_to_pay_or_receive_by_index = float([row.split("€")[1].strip() for row in table.loc[:,0] if "€" in row][0].replace(".","").replace(",","."))
aanslagbiljet_info["Te betalen bedrag"] = amount_to_pay_or_receive_by_index
try:
  aanslagbiljet_info["Rekeningnummer"] = re.search("[A-Z].[0-9].*[0-9]",table.loc[table.iloc[:,0].str.contains("rekeningnummer")].values[0][0].split("rekeningnummer: ")[1])[0]
except IndexError: #Franse klant
  aanslagbiljet_info["Rekeningnummer"] = re.search(re.compile("[A-Z].[0-9].*[0-9]"),table.loc[table.iloc[:,0].str.contains("compte bancaire")].values[0][0])[0]
try:
  aanslagbiljet_info["Mededeling"] = re.split(":.",table.loc[table.iloc[:,0].str.contains("mededeling|communication structurée", regex=True, case=False)].values[0][0])[1].strip()  
  aanslagbiljet_info["Vervaldatum"] = re.search("[0-9].\..*",table.loc[table.iloc[:,0].str.contains("ten laatste|au plus tard le", regex=True, case=False)].values[0][0])[0]
except IndexError: #Terug te krijgen bedrag ipv te betalen
  aanslagbiljet_info["Bedrag in uw voordeel"] = aanslagbiljet_info["Te betalen bedrag"]
  del aanslagbiljet_info["Te betalen bedrag"]

aanslagbiljet_info

{'Mededeling': '+++200/4948/10105+++',
 'Rekeningnummer': 'BE42  679 2 00 00 0 054',
 'Te betalen bedrag': 9951.76,
 'Vervaldatum': '13.01.2021'}